# Uniswap V3 Math Tutorial

In this tutorial, we review the following:
* Swap
* Double-sided withdraw
* Double-sided deposit
* Single-sided withdraw
* Single-sided deposit

In [332]:
from uniswappy import *
import numpy as np
import matplotlib.pyplot as plt
import math

In [333]:
user_nm = 'user_intro'
eth_amount = 1000
dai_amount = 1000000
fee = UniV3Utils.FeeAmount.MEDIUM
tick_spacing = UniV3Utils.TICK_SPACINGS[fee]
lwr_tick = UniV3Utils.getMinTick(tick_spacing)
upr_tick = UniV3Utils.getMaxTick(tick_spacing)

## (1a) Swap derivation: dy -> dx 

In [377]:
eth = ERC20("ETH", "0x09")
dai = ERC20("TKN", "0x111")

exchg_data = UniswapExchangeData(tkn0 = eth, tkn1 = dai, symbol="LP", 
                                   address="0x011", version = 'V3', 
                                   tick_spacing = tick_spacing, 
                                   fee = fee)

factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.deploy(exchg_data)

Join().apply(lp, user_nm, eth_amount, dai_amount, lwr_tick, upr_tick)
lp.summary()

Exchange ETH-TKN (LP)
Real Reserves:   ETH = 1000.0000000000001, TKN = 1000000.0000000001
Gross Liquidity: 31622.776601683796 



Calculate $\Delta x$

$\quad\quad \Delta x = L (\frac{1}{\sqrt{P}} - \frac{1}{\sqrt{P_{next}}})$

where

$\quad\quad \sqrt{P_{next}}  = \sqrt{P} + \frac{997 \Delta y}{1000 L}$

### Perform dy -> dx swap using dervation

In [378]:
dy = 1000

Q96 = 2**96
sqrtp_cur = lp.slot0.sqrtPriceX96/Q96 # convert from Q96 to human

gamma = 997/1000 
x = lp.get_reserve(eth)
y = lp.get_reserve(dai)
L = lp.get_liquidity()

sqrtp_next = sqrtp_cur + (gamma*dy) / (L)
dx = L * (1/sqrtp_cur - 1/sqrtp_next) 

print(f'We receive {dx:.5f} ETH for {dy} DAI')

We receive 0.99601 ETH for 1000 DAI


### Perform dy -> dx swap using uniswappy

In [379]:
out = Swap().apply(lp, dai, user_nm, dy)
lp.summary()

print(f'We receive {out:.5f} ETH for {dy} DAI')
print(f'Confirm price: (1/sqrtp_next^2)={1/sqrtp_next**2:.8f} vs (actual price)={lp.get_price(dai):.8f}') 

Exchange ETH-TKN (LP)
Real Reserves:   ETH = 999.0039930189602, TKN = 1001000.0000000001
Gross Liquidity: 31622.776601683796 

We receive 0.99601 ETH for 1000 DAI
Confirm price: (1/sqrtp_next^2)=0.00099801 vs (actual price)=0.00099801


## (1b) Swap derivation: dx -> dy 

In [364]:
eth = ERC20("ETH", "0x09")
dai = ERC20("TKN", "0x111")

exchg_data = UniswapExchangeData(tkn0 = eth, tkn1 = dai, symbol="LP", 
                                   address="0x011", version = 'V3', 
                                   tick_spacing = tick_spacing, 
                                   fee = fee)

factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.deploy(exchg_data)

Join().apply(lp, user_nm, eth_amount, dai_amount, lwr_tick, upr_tick)
lp.summary()

Exchange ETH-TKN (LP)
Real Reserves:   ETH = 1000.0000000000001, TKN = 1000000.0000000001
Gross Liquidity: 31622.776601683796 



Calculate $\Delta y$

$\quad\quad \Delta y = L (\sqrt{P} - \sqrt{P_{next}})$

where 

$\quad\quad \sqrt{P_{next}}  =\frac{1}{ \frac{1}{\sqrt{P}} + \frac{997 \Delta x}{1000 L}}$

### Perform dx -> dy swap using dervation

In [365]:
dx = 1

Q96 = 2**96
sqrtp_cur = lp.slot0.sqrtPriceX96/Q96 # convert from Q96 to human

gamma = 997/1000 
x = lp.get_reserve(eth)
y = lp.get_reserve(dai)
L = lp.get_liquidity()

sqrtp_next = 1/(1/sqrtp_cur + (gamma*dx)/(L))
dy = L * (sqrtp_cur - sqrtp_next)

print(f'We receive {dy:.5f} DAI for {dx} ETH')

We receive 996.00698 DAI for 1 ETH


In [366]:
out = Swap().apply(lp, eth, user_nm, dx)
lp.summary()

print(f'We receive {out:.5f} DAI for {dx} ETH')
print(f'Confirm price: (sqrtp_next^2)={sqrtp_next**2:.6f} vs (actual price)={lp.get_price(eth):.6f}') 

Exchange ETH-TKN (LP)
Real Reserves:   ETH = 1001.0000000000001, TKN = 999003.9930189601
Gross Liquidity: 31622.776601683796 

We receive 996.00698 DAI for 1 ETH
Confirm price: (sqrtp_next^2)=998.008978 vs (actual price)=998.008978
